In [ ]:
#Secventa de incarcare si dezarhivare a imaginilor
# from google.colab import drive
# drive.mount('/content/gdrive')
# !unzip  gdrive/My\ Drive/unibuc-brain-ad.zip

from PIL import Image
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix

#functia care imi proceseaza imaginile, in functie de ce lungime are numarul imaginii, stiu cati de 0 se afla
#in numele png-ului. folosesc convert("L") pentru a converti imaginile la formatul alb-negru
def process_images(start, end):
  lst = []
  for i in range(start,end):
    if len(str(i)) == 1:
      img = Image.open("data/data/00000" + str(i) + ".png").convert("L")
    elif len(str(i)) == 2:
      img = Image.open("data/data/0000" + str(i) + ".png").convert("L")
    elif len(str(i)) == 3:
      img = Image.open("data/data/000" + str(i) + ".png").convert("L")
    elif len(str(i)) == 4:
      img = Image.open("data/data/00" + str(i) + ".png").convert("L")
    elif len(str(i)) == 5:
      img = Image.open("data/data/0" + str(i) + ".png").convert("L")
    #transform imaginea in array numpy
    img_array = np.array(img)
    #adaug imaginea la lista finala
    lst.append(img_array)
  return lst

#creare imaginile de antrenare, care stiu ca sunt de la 1 la 15000
train_images = np.array(process_images(1,15001))
train_images = train_images.reshape(15000, 50176)

#creare imaginile de validare, in numar de 2000, de la 15001 la 17000
validation_images = np.array(process_images(15001,17001))
validation_images = validation_images.reshape(2000, 50176)

#creare imaginile de testare, 5149 care au mai ramas
test_images = np.array(process_images(17001,22150))
test_images = test_images.reshape(5149, 50176)

def process_labels(start, end, file):
  labels = np.array([])
  with open(file, 'r') as fin:
    #sar peste prima linie cu id,class
    fin.readline()
    for i in range(start,end):
      line = [word for word in fin.readline().strip().split(",")]
      #adaug clasa citita la labels, in format int
      labels = np.append(labels, int(line[1]))
  return labels

#creare labels de antrenare
train_labels = process_labels(1,15001,'/content/data/train_labels.txt')

#creare labels de validare
validation_labels = process_labels(0,2000, '/content/data/validation_labels.txt')

dict_scores = {}

#metoda values_to_bins preluata din laborator unde impart valorile continue in intervale
def values_to_bins(x, bins):
    return np.digitize(x, bins) - 1

#functia care imi ploteaza graficul cu acuratetea in functie de numarul de bins
#asa mi-am dat seama ca folosirea a 4 bins este optima
def create_data():
  #lista unde retin nr de bins-uri incercate
  no_of_bins = []
  #lista unde retin ce acuratete am avut in functie de nr de bins
  accuracies = []

  #testez de la 3 pana la 19 bins
  for i in range(3,20):
    bins = np.linspace(0, 255, num=i)
    no_of_bins.append(i)

    #conversia pixelilor in intervalele in care se afla
    train = values_to_bins(train_images, bins)
    test = values_to_bins(validation_images, bins)

    #instantierea modelului
    naive_bayes_model = MultinomialNB()
    #antrenarea lui
    naive_bayes_model.fit(train, train_labels)
    #adaugarea acuratetei obtinute in lista
    accuracies.append(naive_bayes_model.score(test, validation_labels))
    #asocierea acuratetei obtinute pentru fiecare numar de bins incercat
    dict_scores[i] = accuracies[-1]

  return no_of_bins, accuracies, dict_scores

returned_values = create_data()

#plotez graficul, pe axa Ox am numarul de bins, pe axa Oy acuratetea obtinuta
plt.plot(returned_values[0], returned_values[1])
plt.xlim(returned_values[0][0], returned_values[0][-1])
plt.show()

#in variabila number_bins voi retine numarul de bins pentru care am obtinut acuratea maxima,
#folosindu-ma de dictionarul creat anterior
for cheie in dict_scores:
  if dict_scores[cheie] == max(dict_scores.values()):
    number_bins = cheie
    break

#instantiez modelul
naive_bayes_model = MultinomialNB()

#creare bins
bins = np.linspace(0, 255, num=number_bins)

#conversia indecsilor in intervalele in care se afla
train_images = values_to_bins(train_images, bins)
validation_images = values_to_bins(validation_images, bins)
test_images = values_to_bins(test_images, bins)

#antrenare
naive_bayes_model.fit(train_images, train_labels)

#predictii pentru validation_images pentru crearea matricii de confuzie
pred = naive_bayes_model.predict(validation_images)

#plotarea matricii de confuzie
display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(validation_labels, pred), display_labels = [0,1])
display.plot()
plt.show()

print("Scor: ")
print(metrics.accuracy_score(validation_labels, pred))

print("Precizie clasa 0: ")
print(metrics.precision_score(validation_labels, pred, pos_label=0))

print("Recall clasa 0: ")
print(metrics.recall_score(validation_labels, pred, pos_label=0))

print("Precizie clasa 1: ")
print(metrics.precision_score(validation_labels, pred, pos_label=1))

print("Recall clasa 1: ")
print(metrics.recall_score(validation_labels, pred, pos_label=1))

#functia pentru fisierul de output
def create_output_file(test_images):
  #tot aici obtin si predictiile pentru imaginile de test
  predictions = naive_bayes_model.predict(test_images)
  with open('/content/data/submission.csv', 'w') as fout:
    print("id,class", file = fout)
    #incep de la 17001, prima imagine de test
    image_idx = 17001
    for prediction in predictions:
      print("0"+str(image_idx)+","+str(int(prediction)), file = fout)
      image_idx += 1

create_output_file(test_images)